In [ ]:
#pip install opencage

In [ ]:
from shapely.geometry import Point
from opencage.geocoder import OpenCageGeocode
from geopy.geocoders import Nominatim
import geopandas as gpd
import pandas as pd
import leafmap
import csv
from time import sleep

In [ ]:
API_KEY = "4b904d80060d47be8cb136c12e6823ab"
geocoder = OpenCageGeocode(API_KEY)

def get_geocode(region_or_place):
    try:
        response = geocoder.geocode(region_or_place)
        if response:
            lat = response[0]['geometry']['lat']
            lon = response[0]['geometry']['lng']
            return lat, lon
        return None, None
    except Exception as err:
        print(f"Error while geocoding {region_or_place}: {err}")
        return None, None

input_file = "/content/Regions_places.csv"
output_file = "/content/Geocoded_Regions_places.csv"

with open(input_file, mode='r') as input_csv, open(output_file, mode='w', newline='') as output_csv:
    reader = csv.DictReader(input_csv)

    columns = reader.fieldnames + ['Latitude', 'Longitude']
    writer = csv.DictWriter(output_csv, fieldnames=columns)

    writer.writeheader()

    for entry in reader:
        location = entry.get('Region/Places')
        if location:
            lat, lon = get_geocode(location)
            entry['Latitude'] = lat
            entry['Longitude'] = lon
        else:
            entry['Latitude'] = None
            entry['Longitude'] = None
        writer.writerow(entry)

print(f"Geocoding completed. The results are saved at {output_file}")


Geocoding completed. The results are saved at /content/Geocoded_Regions_places.csv


In [ ]:
icpac_gdf = gpd.read_file('/content/ea_ghcf_simple.json')
data = pd.read_csv('/content/Geocoded_Regions_places.csv')

geometry = [Point(lon, lat) for lon, lat in zip(data['Long'], data['Lat'])]
gdf_points = gpd.GeoDataFrame(data, geometry=geometry)

icpac_gdf = icpac_gdf.to_crs(epsg=4326)
gdf_points = gdf_points.set_crs(epsg=4326, allow_override=True)

gdf_points['inside_icpac'] = gdf_points.geometry.apply(lambda x: icpac_gdf.contains(x).any())

filtered_points = gdf_points[gdf_points['inside_icpac']]

m = leafmap.Map(center=[data['Lat'].mean(), data['Long'].mean()], zoom=6)
m.add_geojson(icpac_gdf.to_json(), layer_name="ICPAC Boundaries")

for _, row in filtered_points.iterrows():
    m.add_marker([row['Lat'], row['Long']])

m


Map(center=[7.9577171423076924, 31.650290787330317], controls=(ZoomControl(options=['position', 'zoom_in_text'…